In [1]:
from langchain_groq import ChatGroq

In [56]:
llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    temperature = 0,
    groq_api_key = "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"
)

In [57]:
response = llm.invoke("Write a log of 700 words on AI")

print(response.content)

**Log Entry: Artificial Intelligence (AI) - A Revolution in Technology**

As we continue to advance in the field of technology, one area that has gained significant attention in recent years is Artificial Intelligence (AI). AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. In this log, we will explore the concept of AI, its history, applications, and the potential impact it may have on our lives.

**History of AI**

The concept of AI dates back to the mid-20th century, when computer scientists such as Alan Turing and Marvin Minsky began exploring the idea of creating machines that could think and learn like humans. The term "Artificial Intelligence" was first coined in 1956 by John McCarthy, a computer scientist who organized the Dartmouth Summer Research Project on Artificial Intelligence. Since then, AI has undergone significant developments, with the creation of t

In [49]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Virat_Kohli")
doc = loader.load().pop()

In [66]:
print(doc.page_content[:3800])





Virat Kohli - Wikipedia


































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Early life








2
Career




Toggle Career subsection





2.1
2008–2009: Debut and maiden stint










3
Player profile




Toggle Player profile subsection





3.1
Comparisons to Sachin Tendulkar








3.2
Playing style








3.3
Aggression










4
Public and media image








5
Personal life








6
Investments & philanthropy




Toggle Investments 

In [52]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("Give the description about this scrapped data : {data}")

chain = template | llm

In [67]:
output = chain.invoke(input={"data": doc.page_content[:3800]})

In [68]:
print(output.content)

The provided data appears to be a Wikipedia article about the Indian cricketer Virat Kohli. The article includes various sections such as:

1. **Early life**: Although the section is not fully provided, it is likely to contain information about Kohli's birth, family, and upbringing.
2. **Career**: This section is also not fully provided, but it seems to cover Kohli's debut and early years in cricket.
3. **Player profile**: This section includes subsections on:
	* **Comparisons to Sachin Tendulkar**: Likely discussing how Kohli is often compared to the legendary Indian cricketer Sachin Tendulkar.
	* **Playing style**: Describing Kohli's batting and bowling techniques.
	* **Aggression**: Possibly discussing Kohli's aggressive playing style and behavior on the field.
4. **Public and media image**: This section might cover Kohli's reputation and how he is perceived by the public and media.
5. **Personal life**: Although not fully provided, this section likely includes information about Koh

In [71]:
import chromadb
chroma_client = chromadb.Client()


ModuleNotFoundError: No module named 'chromadb'